In [1]:
machado_path = 'kaggle/input/machado-de-assis/machado.txt'
with open (machado_path, 'r', encoding='utf-8') as f:
    machado = f.read()

print("Length of Machado's text: {} characters".format(len(machado)))

Length of Machado's text: 10977785 characters


We are working with 10 million characters that in some way encode the manner that machado writes it texts.

In [2]:
chars = sorted(list(set(machado)))
vocab_size = len(chars)
print(''.join(chars))
print('There are {} unique characters in the Machado corpus'.format(vocab_size))


 !"$%&'()*+,-./0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyz§ª°ÀÁÂÃÇÈÉÊËÍÓÔÕÚÛÜàáâãäçèéêëìíîïñòóôõöùúûüœ
There are 127 unique characters in the Machado corpus


##### Building basic Encoder and Decoder
Encoder get a string and map each character to an integer. <br>
Decoder get a list of integers and map each integer to a character

This is the way that we choose to tokenize our pieces of word. <br>
Google uses "SentencePiece" and OpenAi uses "tiktoken". <br>
Those encoders have a different vocabulary size.

In [3]:
stoi = {ch:i for i,ch in enumerate(chars)} # dictionary mapping characters to integers
itos = {i:ch for i,ch in enumerate(chars)} # dictionary mapping integers to characters
encode = lambda s: [stoi[ch] for ch in s] # encode string to list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decode list of integers to string

print(encode("Machado de Assis"))
print(decode(encode("Machado de Assis")))

[42, 57, 59, 64, 57, 60, 71, 1, 60, 61, 1, 30, 75, 75, 65, 75]
Machado de Assis


##### Using pytorch

In [4]:
import torch
machado_data = torch.tensor(encode(machado), dtype=torch.long)
print(machado_data.shape, machado_data.dtype)

torch.Size([10977785]) torch.int64


##### Spliting data
Splitting data into train and validation

In [5]:
n = int(len(machado_data) * 0.7)
train_data = machado_data[:n]
val_data = machado_data[n:]

##### Using Batches and DataLoader

In [6]:
#torch.manual_seed(42)

def get_batch(split):
    # split can be 'train' or 'val'
    # generate small random batch of data of input x and target y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])     # create an x matrix of size (batch_size, block_size)
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # create an y matrix of size (batch_size, block_size)
    x, y = x.to(device), y.to(device)
    return x, y

##### Estimating Loss

We do this because as we are working with batches, we can get more lucky in some bathes than others.

In [7]:
import torch.nn as nn
from torch.nn import functional as F

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

#### Constructing the model structure

Classes: <br>
- Head
- MultiHeadAttention
- FeedForward
- Block

In [8]:
class Head(nn.Module):
    ''' one head of self-attention mechanism'''

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input x has shape (batch_size, time-step, channels)
        # output has shape (batch_size, time-step, head_size)
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        
        # compute attention weights
        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        
        # perform attention of the values
        v = self.value(x)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        
        return out

class MultiHeadAttention(nn.Module):
    ''' multi-head attention mechanism'''

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    ''' simple linear layer followed by a non-linear activation function'''

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    ''' transformer block'''

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd) 
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):   
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

#### Creating Model

In [9]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)   
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.ln_head = nn.Linear(n_embd, vocab_size) # final linear layer

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        tok_emb = self.token_embedding_table(idx) # (B, T) -> (B, T, n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T) -> (T, n_embd)
        x = tok_emb + pos_emb # (B, T, n_embd)
        x = self.blocks(x) # (B, T, n_embd)
        x = self.ln_f(x) # (B, T, n_embd)
        logits = self.ln_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            #print(logits.shape, targets.shape)
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            #print(logits.shape, targets.shape)
            loss = F.cross_entropy(logits, targets) #, ignore_index=0

        return logits, loss

    def generate(self, idx, max_new_tokens):
        ''' generate new tokens from the given tokens'''
        
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx 

#### Setting up Hyperparameters:

In [10]:
batch_size = 64 # how many independent sequences will we process in parallel / number of sequences in a batch (we can process batches in parallel) 
block_size = 256 # what is the maximum context length for predictions? / length of a sequence (maximum context lengt)
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
print(torch.zeros(1).cuda())

tensor([0.], device='cuda:0')


#### Instantiating Model

In [11]:
model = GPTLanguageModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(sum(p.numel() for p in model.parameters() if p.requires_grad), 'parameters')

10836607 parameters


In [12]:
for iter in range(max_iters):

    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate text from the model
context = torch.zeros((1,1), dtype=torch.long, device=device) # start with a single <BOS> token      
     

torch.Size([64, 256, 127]) torch.Size([64, 256])
torch.Size([64, 256, 127]) torch.Size([16384])


ValueError: Expected input batch_size (64) to match target batch_size (16384).

In [ ]:
print(decode(m.generate(context, max_new_tokens=1000).[0].tolist()))